In [40]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import pickle


In [41]:
# Load the dataset
data=pd.read_csv('Churn_Modelling.csv')
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [42]:
# Drop irrelevant columns
data.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1, inplace=True)
data.head()


,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [43]:
# Encode the categorical columns
label_encoder_gender = LabelEncoder()
data['Gender'] = label_encoder_gender.fit_transform(data['Gender'])
data.head()


,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0


In [44]:
# One hot encoding of Geography column
from sklearn.preprocessing import OneHotEncoder
onehot_encoder_geo=OneHotEncoder()
geo_encoder=onehot_encoder_geo.fit_transform(data['Geography'].values.reshape(-1,1)).toarray()
geo_encoder

array([[1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       ...,
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.]])

In [45]:
# Feature Names
onehot_encoder_geo.get_feature_names_out(['Geography'])

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [46]:
# Convert into DataFrame4
geo_encoded_df=pd.DataFrame(geo_encoder, columns=onehot_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df.head()

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0


In [47]:
# covert the Geography column into one hot encoded columns
data=pd.concat([data.drop('Geography',axis=1), geo_encoded_df], axis=1)
data.head()


,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [48]:
# save the encoder
with open('label_encoder_gender.pkl', 'wb') as file:
    pickle.dump(label_encoder_gender, file)

# save the one hot encoder
with open('onehot_encoder_geo.pkl', 'wb') as file:
    pickle.dump(onehot_encoder_geo, file)

In [49]:
# Divide the dataset into dependent and independent variables
X=data.drop('Exited', axis=1)
y=data['Exited']

# split the dataset into training and testing
X_train, X_test, y_train, y_test=train_test_split(X,y, test_size=0.2, random_state=42)

# Feature Scaling
sc=StandardScaler()
X_train=sc.fit_transform(X_train)
X_test=sc.transform(X_test)

In [50]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((8000, 12), (2000, 12), (8000,), (2000,))

In [51]:
# save the standard scaler
with open('sc.pkl', 'wb') as file:
    pickle.dump(sc, file)

## ANN Implementation

In [52]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
import datetime

In [53]:
type((X_train.shape[1],))

tuple

In Below Code in second Dense I haven't use Input_shape as we know that as we have used sequential all these layers are interconnected. Also, in last dense I have used units=1 as The o/p is binary and thus we are expected one o/p neuron and thus sigmoid used as the activation function

In [54]:
# Build the ANN model
model=Sequential([Dense(units=64, activation='relu', input_shape=(X_train.shape[1],)), ## HL1 connected to input layer
                 Dense(units=32, activation='relu'), ## HL2 connected to HL1
                 Dense(units=1, activation='sigmoid')]) ## Output layer




In [55]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 64)                832       
                                                                 
 dense_4 (Dense)             (None, 32)                2080      
                                                                 
 dense_5 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


## Above summary will tell us that there are 3 layers HL1,HL2 & O/P and total number of hiddem neuron in each layer.Also,# of parameter given is actually showing comibination of weights and the bias in each layers

This is another way to give above parameters
import tensorflow
opt=tensorflow.keras.optimizers.Adam(learning_rate=0.01)
loss=tensorflow.keras.losses.BinaryCrossentropy(from_logits=True)

In [56]:
#compile the model to do forwardpropagation backpropagation and update the weights
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [57]:
## Setups for Tensorboard for visualization of the log generated after model training
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

## early stopping : Basically we monito the loss value and performance of the model while training the 
ANN model, after certain EPOCHs, if these are not changing significantly then we can call back EARLYSTOPPING TO EXIT between the designated number of EPOCHs. Like out of 100 EPOCHs we mentioned we saw this at 20 EPOC then we don't need to run rest 80 EPOCHs we will exit the process
Early stopping is a regularization technique used in training artificial neural networks (ANNs) to prevent overfitting. Overfitting occurs when a model learns the training data too well, including its noise and outliers, which negatively impacts its performance on new, unseen data.

How Early Stopping Works
1.Training and Validation Sets: The dataset is split into training and validation sets. The model is trained on the training set, and its performance is periodically evaluated on the validation set.

2.Monitoring Performance: During training, the model's performance on the validation set is monitored. Typically, metrics like validation loss or accuracy are tracked.

3. Stopping Criteria: If the performance on the validation set stops improving for a specified number of epochs (patience), training is halted. This indicates that the model has likely reached its optimal point and further training would lead to overfitting.

In [58]:
## set up early stopping to stop the model training if the model is not learning
early_stopping_callback=EarlyStopping(monitor='val_loss', patience=10,restore_best_weights=True)

In [59]:
## Train the model ANN
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=100, callbacks=[tensorflow_callback, early_stopping_callback])


Epoch 1/100
250/250 [==============================] - 1s 2ms/step - loss: 0.4612 - accuracy: 0.7896 - val_loss: 0.3910 - val_accuracy: 0.8290
Epoch 2/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3816 - accuracy: 0.8435 - val_loss: 0.3544 - val_accuracy: 0.8515
Epoch 3/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3545 - accuracy: 0.8558 - val_loss: 0.3477 - val_accuracy: 0.8565
Epoch 4/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3449 - accuracy: 0.8572 - val_loss: 0.3502 - val_accuracy: 0.8585
Epoch 5/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3385 - accuracy: 0.8618 - val_loss: 0.3458 - val_accuracy: 0.8555
Epoch 6/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3357 - accuracy: 0.8627 - val_loss: 0.3539 - val_accuracy: 0.8540
Epoch 7/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3330 - accuracy: 0.8615 - val_loss: 0.3453 - val_accuracy: 0.8590

In [60]:
## Save the model file compatible with keras
model.save('model.h5')

c:\Users\Ankit Ghai\Documents\PycharmProjects\pythonProject\ANNClassification\venv\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [61]:
## Load tensorboard extension
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [62]:
%tensorboard --logdir logs/fit/20241102-184101

Reusing TensorBoard on port 6006 (pid 4380), started 1:34:33 ago. (Use '!kill 4380' to kill it.)